In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime
%matplotlib inline


In [ ]:
prop =pd.read_csv('../input/properties_2016.csv')
prop.shape

In [ ]:
prop.head()

In [ ]:
train =pd.read_csv('../input/train_2016_v2.csv', parse_dates=['transactiondate']) 
## parse dates, otherwise date appears as object and need to be converted
train.shape


In [ ]:
train.head()

## Questions in the mind (Data Exploration):
---
(Edited as an when i figure out thing)
* What is the target variable? - *log Error in train*.
    - Continuous or discrete? - *continuous [-4,4]*
    - If discrete what is the range? Skewed distribution?
    - Log variables
* How are the two data sets related?
    - What is the primary key? - *using parcelid alone*
* Types of columns in training dataset? All float/string/categorical objects ? - *yes. all 3.*
* Data Transformation
    - In right scale or need transformation like log or exp?
* Amount of Null Values
    - Columns with all/almost most  values Null
    - Null value handling
* How are some of the variables related to target variable? (Univariate Analysis)
   - Top 20 impacting variables
   - Top +ve or -ve correlated variables
* How are the variables related among themselves? (Bi variate) --> To avoid correlated variables
* How are they related to Target variable (Multi variate)
* Try different models and see their performance

In [ ]:
prop.head()

## Exploring train dataset

**1. Parcelid column**

In [ ]:
train.parcelid.value_counts().head()

In [ ]:
#(train.parcelid.value_counts().reset_index())['parcelid'].head()
(train.parcelid.value_counts().reset_index())['parcelid'].value_counts()
## Most parcelids in train data are unique. only 124 items repeat

**2. logerror column exploration**

In [ ]:
train.describe() ## target var: logerror, continous [-4.6,4.7]. Q1,Q3-[-0.02,.03]

In [ ]:
plt.figure(figsize=(12,5))
plt.hist(train.logerror.values,bins=50,range=(-1,1))
plt.xlabel('logerror')
plt.ylabel('frequency')
plt.title('Distribution of log error')
plt.show()

Actually the target variable shows normal distribution, except on the extremes, with  tail on either side

**3. transaction date column exploration**

In [ ]:
train.transactiondate.value_counts().head() ## idea abt what dates are present

In [ ]:
plt.hist(train.transactiondate.values, bins=366) ## just for an year
plt.xlabel("Transaction date")
plt.ylabel("freq")
plt.show()

Two things: 
1. #transactions increased as we progress in the year ( #transactions vs month might help).
2. Some days of have high transactions and some days might have low transactions ( #transactions vs day of the week or day of the month)

In [ ]:
train['transactionmonth']=train.transactiondate.dt.month.values
month_df = train['transactionmonth'].value_counts()
#plt.hist(x=month_df.index, y=month_df.values) ## Not working

plt.figure(figsize=(12,5))
plt.bar(month_df.index,month_df.values) 

plt.xlabel("Transaction month")
plt.ylabel("freq")
plt.title('Distribution of transactions over different months')
plt.show()
#month_df

Data present for first 10 months. some sample data for oct,nov,dec.

In [ ]:
train['transactionday']=train.transactiondate.dt.day.values
day_df = train['transactionday'].value_counts()
#plt.hist(x=month_df.index, y=month_df.values) ## Not working

plt.figure(figsize=(12,5))
plt.bar(day_df.index,day_df.values) 

plt.xlabel("Transaction day")
plt.ylabel("freq")
plt.title('Distribution of transactions over different days of the month')
plt.show()
#month_df


No real pattern based in the dates. Beginning and end of the month may be higher transactions, and not many months have 31days

In [ ]:
train['transactiondayofweek']=train.transactiondate.dt.dayofweek.values
transactiondayofweek_df = train['transactiondayofweek'].value_counts()
#plt.hist(x=month_df.index, y=month_df.values) ## Not working

plt.figure(figsize=(12,5))
plt.bar(transactiondayofweek_df.index,transactiondayofweek_df.values) 

plt.xlabel("Transaction day of the week")
plt.ylabel("freq")
plt.title('Distribution of transactions over different days of the week ( Monday=0, Sunday=6)')
plt.show()

Increase in number of transactions in the due course of the week. Friday higher number of transactions

## Prop table exploration

In [ ]:
prop.head()

In [ ]:
prop.dtypes.value_counts()

In [ ]:
prop.describe()

Basementsqft, yardbuildingsqft26, numberofstories, structure tax value dollarcnt, taxvaluedollarcnt, taxamount might require some transformation. Continuous values.

In [ ]:
## Merging data
train_prop = pd.merge(train,prop,how='left',on='parcelid')
train_prop.head()


In [ ]:
## Sample model
train_prod = pd.merge(train,prop,   how='left', on='parcelid')
median_val = train_prod.median(axis=0)
train_prod = train_prod.fillna(median_val,inplace=True)
train_y=train_prod.logerror

cat_cols = ["hashottuborspa", "propertycountylandusecode", "propertyzoningdesc", "fireplaceflag", "taxdelinquencyflag"]
train_x= train_prod.drop(['parcelid','logerror','transactionday','transactiondate','transactiondayofweek','transactionmonth']+cat_cols,axis=1)


In [ ]:
test_df = prop
test_x = test_df.drop(cat_cols,axis=1)

In [ ]:
import xgboost as xgb
xgb_params = {
    'eta': 0.05,
    'max_depth': 8,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'silent': 1,
    'seed' : 0
}

dtrain= xgb.DMatrix(train_x, train_y, feature_names= train_x.columns.values)
model=xgb.train(dict(xgb_params),dtrain, num_boost_round=100,verbose_eval=10)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
xgb.plot_importance(model,max_num_features=20,title='Feature Importance',ax=ax,height=0.5)
plt.show()